In [4]:
import os
import sys

# run locally without install asynfed package
root = os.path.dirname(os.path.dirname(os.path.dirname(os.path.dirname(os.getcwd()))))
sys.path.append(root)


In [14]:
# tensorflow 
from asynfed.client.frameworks.tensorflow.tensorflow_framework import TensorflowFramework
from resnet18 import Resnet18
from data_preprocessing import *


In [15]:
batch_size = 128
epoch = 5

In [16]:

# load data
default_training_dataset_path = "../../../data/cifar_data/5_chunks/chunk_2.pickle"
default_testing_dataset_path = "../../../data/cifar_data/test_set.pickle"


# train_ds, data_size = preprocess_dataset(training_dataset_path, training = True)
# test_ds, _ = preprocess_dataset(testing_dataset_path, training = False)
train_ds, data_size = preprocess_dataset(default_training_dataset_path, batch_size = 128, training = True)
test_ds, _ = preprocess_dataset(default_testing_dataset_path, training = False)


2023-07-02 05:24:53.276978: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1635] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22301 MB memory:  -> device: 0, name: NVIDIA RTX A5000, pci bus id: 0000:19:00.0, compute capability: 8.6
2023-07-02 05:24:53.277493: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1635] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 22301 MB memory:  -> device: 1, name: NVIDIA RTX A5000, pci bus id: 0000:8d:00.0, compute capability: 8.6


In [17]:

# define model
model = Resnet18(input_features = (32, 32, 3), output_features = 10, lr=1e-1, decay_steps=int(epoch * data_size / batch_size))
# define framework
tensorflow_framework = TensorflowFramework(model = model, epoch= 200, data_size= data_size, train_ds= train_ds, test_ds= test_ds, regularization='l2', delta_time= 10000, qod= 0.45)



In [18]:
# fit to be able to load model
for images, labels in tensorflow_framework.train_ds:
    train_acc, train_loss= tensorflow_framework.fit(images, labels)
    break


2023-07-02 05:24:57.688424: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype float and shape [4166,10]
	 [[{{node Placeholder/_1}}]]
2023-07-02 05:24:57.688679: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype double and shape [4166,32,32,3]
	 [[{{node Placeholder/_0}}]]
2023-07-02 05:25:00.955560: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:424] Loaded cuDNN version 8600
2023-07-02 05:25:01.932945: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:637] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2

In [19]:
file_name = "weights.pkl"
with open(file_name, "rb") as f:
    import pickle
    weights = pickle.load(f)

tensorflow_framework.set_weights(weights)

In [20]:
tensorflow_framework.model.test_loss.reset_states()
tensorflow_framework.model.test_performance.reset_states()

for test_images, test_labels in tensorflow_framework.test_ds:
    test_acc, test_loss = tensorflow_framework.evaluate(test_images, test_labels)

print("Test Acc {:.2f}  Test Loss {}".format(test_acc  * 100, test_loss) )


2023-07-02 05:25:16.174491: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype float and shape [10000,10]
	 [[{{node Placeholder/_1}}]]


Test Acc 95.17  Test Loss 0.1761750727891922
